In [2]:
from sklearn.svm import SVC
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import xarray as xr
from numpy.linalg import norm
from scipy.interpolate import interp1d


def boundary_from_contour_with_gradients_2D(xx, yy, Z, boundary_resolution_len: float):
    plt.ioff()  # Turn interactive mode off to suppress showing plots

    # Extract decision boundary points using contour
    contours = plt.contour(xx, yy, Z, levels=[0], linewidths=1, colors="black")
    plt.close()

    # Get all paths
    paths = contours.allsegs[0]  # Get all segments for the contour level

    all_interpolated_points = []
    all_gradients = []

    for path_segment in paths:
        vertices = np.array(path_segment)
        distances = norm(vertices[1:] - vertices[:-1], axis=1)
        cumulative_distances = np.cumsum(distances)
        cumulative_distances = np.insert(cumulative_distances, 0, 0)

        # Interpolate points along the path
        path_length = cumulative_distances[-1]
        num_points = int(path_length // boundary_resolution_len) + 1
        evenly_spaced_distances = np.linspace(0, path_length, num=num_points)

        interp_fn = interp1d(cumulative_distances, vertices, axis=0, kind='linear')
        interpolated_points = interp_fn(evenly_spaced_distances)

        if len(interpolated_points) < 2:
            # Add a dummy gradient (e.g., zero) if only one point
            gradients = np.zeros_like(interpolated_points)
        else:
            # Compute gradient vectors (finite difference method)
            gradients = np.zeros_like(interpolated_points)
            for i in range(1, len(interpolated_points) - 1):
                tangent = interpolated_points[i + 1] - interpolated_points[i - 1]
                normal = np.array([-tangent[1], tangent[0]])  # Perpendicular vector
                normal /= norm(normal)  # Normalize the gradient
                gradients[i] = normal * 0.1  # Scale the gradient (optional)

            # Set boundary gradients for the endpoints
            gradients[0] = gradients[1]
            gradients[-1] = gradients[-2]

        # Store points and gradients
        all_interpolated_points.append(interpolated_points)
        all_gradients.append(gradients)



        # Store points and gradients
        all_interpolated_points.append(interpolated_points)
        all_gradients.append(gradients)

    return all_interpolated_points, all_gradients


def plot_2D_boundary(X, y, xx, yy, Z, interp_boundary_points, boundary_gradients):
    # Plot the decision boundary and data points
    plt.figure(figsize=(8, 6))
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', s=5, label="Data Points")
    plus_minus = 0.1
    # plt.contour(xx, yy, Z, levels=[-plus_minus,0,plus_minus], linestyles=['-'], colors='k')
    plt.contour(xx, yy, Z, levels=[0,plus_minus], linestyles=['-'], colors='k')

    # Plot interpolated points and gradient vectors
    # for i, interpolated_points in enumerate(interp_boundary_points):
    #     plt.plot(interpolated_points[:, 0], interpolated_points[:, 1], 'go', markersize=5, label=f'Boundary Points (Path {i+1})')



    # Plot boundary points and gradient vectors
    for i, (interpolated_points, gradients) in enumerate(zip(interp_boundary_points, boundary_gradients)):
        plt.plot(interpolated_points[:, 0], interpolated_points[:, 1], 'go', markersize=5)
        plt.quiver(
            interpolated_points[:, 0], interpolated_points[:, 1],
            gradients[:, 0]*.005, gradients[:, 1]*.005,
            angles='xy', scale_units='xy', scale=0.01, color='r', alpha=0.8, headwidth=3
        )

    # Add plot details
    plt.title("SVM Decision Boundary with Gradient Vectors")
    plt.xlim(X[:, 0].min(), X[:, 0].max())
    plt.ylim(X[:, 1].min(), X[:, 1].max())
    plt.xlabel("x0")
    plt.ylabel("x1")
    plt.legend()
    plt.show()


def svm_boundary_from_xarray_2D(dataset:xr.Dataset, param_grid, boundary_resolution_len=0.5, boundary_mesh_resolution = 500):
    """
    Fits an SVM model to the data in the xarray.Dataset and plots the decision boundary.

    Parameters:
    - dataset (xarray.Dataset): The input dataset containing 'points' and 'noise_values'.

    Returns:
    - None: Plots the SVM decision boundary and data points.
    """

    # Ensure the required variables are present
    if not {'points', 'noise_values'}.issubset(dataset.variables):
        raise ValueError("Dataset must contain 'points' and 'noise_values' variables.")

    # Extract features and labels
    X = dataset['points'].values  # Shape (num_points, dim)
    y = dataset['noise_values'].values  # Shape (num_points,)

    # Perform grid search to find the best parameters
    svc = SVC(kernel='rbf')
    grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X, y)

    # Get the best model
    best_svm = grid_search.best_estimator_
    print(f"Best parameters: {grid_search.best_params_}")


    # Create a grid for decision boundary visualization
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, boundary_mesh_resolution),
                        np.linspace(y_min, y_max, boundary_mesh_resolution))

    # Evaluate the decision function
    Z = best_svm.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # interp_boundary_points = boundary_from_contour(xx, yy, Z, boundary_resolution_len)


    interp_boundary_points, boundary_gradients = boundary_from_contour_with_gradients_2D(xx, yy, Z, boundary_resolution_len)


    # Ensure 2D data for visualization
    # if X.shape[1] == 2:
    #     plot_2D_boundary(X, y, xx, yy, Z, interp_boundary_points, boundary_gradients)

    return best_svm, grid_search.best_params_


In [3]:

from src.data_generators.perlin_generator import PerlinNoiseGenerator
from src.data_samplers.random_sampler import sample_xarray
from src.edge_detection.SVM_boundary import SVMBoundary
# Initialize the generator with 2D resolution

size = 50
sample_fracs = [.1, .2, .3, .4, .5, .6, .7]
seeds = [1,2,3,4,5,6,7,8,9,10]
octs = [1,2, 3,4,5,6,7,8,9,10]
each_oct_params = []
each_oct_errors = []


# Define the parameter grid for grid search
param_grid = {
    'C': [0.01, 0.1, 1, 5, 10, 50, 100, 200, 300],  # Regularization parameter
    'gamma': [0.001, 0.1, 0.5, 1, 5, 10, 50, 100]  # Kernel coefficient
}

for oct in octs:
    print("oct ", oct)
    params = []
    errors = []
    for sample_frac in sample_fracs:
        print("sample_frac ", sample_frac)
        temp_params = []
        temp_errors = []
        for seed in seeds:
            generator_2D = PerlinNoiseGenerator(res=[size, size], octv=oct, seed=seed, time_test=False)

            # Generate Perlin noise data
            xdataset_2D = generator_2D.perlin_data_xarray()

            svm_boundary, param = svm_boundary_from_xarray_2D(sample_xarray(xdataset_2D, int(sample_frac*size**2)), param_grid, boundary_resolution_len=0.02)
            temp_params.append(param)
            # generator_2D.plot_2D_xarray()

             # Reshape noise values to grid format for visualization
            noise_values = xdataset_2D['noise_values'].values.reshape(size, size)  # Reshape to grid
            points_x = xdataset_2D['points'].values[:, 0].reshape(size, size)  # x-coordinates reshaped
            points_y = xdataset_2D['points'].values[:, 1].reshape(size, size)  # y-coordinates reshaped

            X = xdataset_2D['points'].values  # Shape (num_points, dim)
            # y = dataset['noise_values'].values  # Shape (num_points,)

            # Create a grid of points in the input space
            x_min, x_max = X[:, 0].min(), X[:, 0].max()
            y_min, y_max = X[:, 1].min(), X[:, 1].max()
            xx, yy = np.meshgrid(np.linspace(x_min, x_max, size), np.linspace(y_min, y_max, size))
            grid_points = np.c_[xx.ravel(), yy.ravel()]

            # Predict classifications on the grid
            predictions = svm_boundary.predict(grid_points)
            predictions = predictions.reshape(xx.shape)
            predictions = np.flipud(predictions)  # Flip along both axes
            predictions = np.rot90(predictions, k=3)

            # # Create a figure with 3 subplots
            # fig, axes = plt.subplots(1, 3, figsize=(6, 4))  # 3 rows, 1 column of subplots
            #
            # # First plot: SVM Classification Regions
            # axes[0].contourf(points_x, points_y, predictions, alpha=0.8, cmap='coolwarm')
            # axes[0].set_title("SVM Classification Regions")
            # axes[0].set_xlabel("x0")
            # axes[0].set_ylabel("x1")
            #
            # # Second plot: True Noise Values
            # axes[1].contourf(points_x, points_y, noise_values, alpha=0.8, cmap='coolwarm')
            # axes[1].set_title("True Noise Values from Perlin Generator")
            # axes[1].set_xlabel("x0")
            # axes[1].set_ylabel("x1")

            # Compute the difference
            difference = noise_values - predictions

            # # Third plot: Difference
            # axes[2].contourf(points_x, points_y, difference, alpha=0.8, cmap='coolwarm')
            # axes[2].set_title("Difference Between Dense Dataset and SVM Predictions")
            # axes[2].set_xlabel("x0")
            # axes[2].set_ylabel("x1")
            #
            # # Adjust layout
            # plt.tight_layout()
            # plt.show()

            # Compute absolute differences
            abs_diff = np.abs(difference)

            # Sum all differences to get the total score
            total_score = np.sum(abs_diff)

            # Optional: Normalize the score by the number of elements
            normalized_score = total_score / difference.size
            temp_errors.append(normalized_score)
        errors.append(sum(temp_errors)/len(temp_errors))
        # params.append(temp_params/temp_params)
    each_oct_errors.append(errors)
    each_oct_params.append(params)

for i in range(len(octs)):
    plt.plot(sample_fracs, each_oct_errors[i], label=f"Oct {octs[i]}")
plt.title("Errors per Sample Percent by Octave")
plt.legend()
plt.xlabel("Sample Fraction")
plt.ylabel("Error")
plt.show()

# plt.plot(octs, [param['C'] for param in params])
# plt.title("SVM Parameter C")
# plt.show()
# plt.plot(octs, [param['gamma'] for param in params])
# plt.title("SVM Parameter Gamma")
# plt.show()

oct  1
sample_frac  0.1
Best parameters: {'C': 200, 'gamma': 10}
Best parameters: {'C': 5, 'gamma': 50}
Best parameters: {'C': 100, 'gamma': 10}
Best parameters: {'C': 5, 'gamma': 5}
Best parameters: {'C': 10, 'gamma': 10}
Best parameters: {'C': 100, 'gamma': 5}


KeyboardInterrupt: 